In [134]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from random import randrange

In [2]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

s = requests.Session()
s.headers.update(headers)


#s.get('https://www.alpsonline.org/reservation/calendar?hut_id=27')
#r = s.get('https://www.alpsonline.org/reservation/selectDate?date=10.07.2024')
#json_list = []
#for i, value in r.json().items():
#    json_list.append(value[0])
#df = pd.DataFrame(json_list)

In [8]:
hutid = [27, 515]
date = '10.07.2024'

#loop over all huts, create a dataframe for each hut
#initialize dataframes
df_list = []

for hut in hutid:
    r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
    r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
    json_list = []
    for i, value in r.json().items():
        json_list.append(value[0])
    df = pd.DataFrame(json_list)
    df['hutid'] = hut
    df_list.append(df)


#reduce dataframes to relevant columns, add hutid and calculate occupation
df_red_list = []
for df in df_list:
    df_red = df[['hutid', 'reservationDate', 'freeRoom']].copy()
    df_red['occupation'] = (100 - df_red['freeRoom'].copy() / df['totalRoom'].copy() * 100)
    df_red['Service'] = df['bedCategoryType'].copy()
    df_red_list.append(df_red)

#concatenate dataframes
df_con = pd.concat(df_red_list)
print(df_con)


    hutid reservationDate  freeRoom  occupation     Service
0      27      14.07.2024        36   32.075472        ROOM
1      27      15.07.2024        28   47.169811        ROOM
2      27      16.07.2024        24   54.716981        ROOM
3      27      17.07.2024        28   47.169811        ROOM
4      27      18.07.2024        40   24.528302        ROOM
5      27      19.07.2024         7   86.792453        ROOM
6      27      20.07.2024         1   98.113208        ROOM
7      27      21.07.2024        36   32.075472        ROOM
8      27      22.07.2024        27   49.056604        ROOM
9      27      23.07.2024        21   60.377358        ROOM
10     27      24.07.2024        45   15.094340        ROOM
11     27      25.07.2024        46   13.207547        ROOM
12     27      26.07.2024        35   33.962264        ROOM
13     27      27.07.2024         0  100.000000        ROOM
0     515      14.07.2024         6   40.000000  UNSERVICED
1     515      15.07.2024        10    0

In [ ]:
#Read csv file with hut information
#contains: hutid, hutname, altitude
df_huts = pd.read_excel('hutInfo.xlsx')

In [12]:
#look up reservations

hutid = 680
date = '10.07.2024'

r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
json_list = []
for i, value in r.json().items():
    json_list.append(value[0])
df = pd.DataFrame(json_list)
df['hutid'] = hut
df_list.append(df)

print(df)

    hutBedCategoryId  bedCategoryId bedCategoryType  bookingEnabled  closed  \
0               1350             43      UNSERVICED            True   False   
1               1350             43      UNSERVICED            True   False   
2               1350             43      UNSERVICED            True   False   
3               1350             43      UNSERVICED            True   False   
4               1350             43      UNSERVICED            True   False   
5               1350             43      UNSERVICED            True   False   
6               1350             43      UNSERVICED            True   False   
7               1350             43      UNSERVICED            True   False   
8               1350             43      UNSERVICED            True   False   
9               1350             43      UNSERVICED            True   False   
10              1350             43      UNSERVICED            True   False   
11              1350             43      UNSERVICED 

In [187]:
def findHutIDs(SACOnly = False):
    #goes through all hut ids and checks if the hut exists
    #if the hut exists, it saves the hut id, altitude and coordinates in a dataframe
    #if the hut does not exist, it is skipped
    #returns a dataframe with hut information
    hutInfo = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Coordinates', 'Reservation Link'])

    date = '11.07.2027' #pick a date well into the future
    hutCounter = 0
    hutidLimit = 5  #up to which hut we should check

    for hutid in range(1,hutidLimit):
        #wait a bit to not overload the server
        time.sleep(0.5+ 0.1*randrange(0,10))
        page = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en')
        soup = bs(page.content, 'html.parser')
        
        #Check if the hut exists
        erm = soup.find_all('div', {'class': 'errorsMessage'})
        #check if a hut exists for the given hutid
        noHutError = f'<div class="errorsMessage">The requested hut [{hutid}] cannot be found. Please check your parameters.</div>'
        hutNotActivatedError = f'<div class="errorsMessage">This hut profile is not activated. Bookings cannot be entered at the moment. Please contact the administrator.</div>'
        if str(erm[3]) == noHutError:
            print(f'Hut with id {hutid} not found')
        elif str(erm[3]) == hutNotActivatedError:
            print(f'Hut with id {hutid} not activated')
        else:
            print(f'Hut with id {hutid} found')
            hutInfBS = soup.body.find('div', attrs={'class': 'info'}).find_all('span')
            hutname = soup.body.find('div', {'class': 'info'}).find_all('h4')[0].text
            if SACOnly:
                if 'SAC' in hutname:
                    hutInfo.loc[hutCounter] = [hutid, hutname, hutInfBS[3].text[24:-2], hutInfBS[4].text[13:], f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en']
                    hutCounter += 1
            else:
                hutInfo.loc[hutCounter] = [hutid, hutname, hutInfBS[3].text[24:-2], hutInfBS[4].text[13:], f'https://www.alpsonline.org/reservation/calendar?hut_id={hutid}&selectDate={date}&lang=en']
                hutCounter += 1


    return hutInfo

Hut with id 1 found
Hut with id 2 found
Hut with id 3 found
Hut with id 4 found
   Hut ID                                      Hut Name Altitude  \
0       1  Blüemlisalphütte SAC, SAC Sektion Blümlisalp     2840   
1       2             Glecksteinhütte SAC, SAC Burgdorf     2317   
2       3      Capanna da l'Albigna CAS, SAC Hoher Rohn     2336   
3       4                    Kesch-Hütte SAC, SAC Davos     2625   

         Coordinates                                   Reservation Link  
0  625.565 / 151.040  https://www.alpsonline.org/reservation/calenda...  
1  650.400 / 163.979  https://www.alpsonline.org/reservation/calenda...  
2  770.675 / 133.375  https://www.alpsonline.org/reservation/calenda...  
3  786.561 / 168.870  https://www.alpsonline.org/reservation/calenda...  


'https://www.alpsonline.org/reservation/calendar?hut_id=1&selectDate=11.07.2027&lang=en'

In [179]:
#check if the string "SAC" is present in the string a
if "SAC" in a:
    print("SAC found")
else:
    print("SAC not found")

SAC found
